##### SIGINATE 【練習問題】自動車の走行距離予測
##### https://signate.jp/competitions/121

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df_train = pd.read_csv('train.tsv',delimiter='\t' )
df_train.head()

,id,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,0,29.0,4,135.0,84.00,2525.0,16.0,82,1,dodge aries se
1,3,31.9,4,89.0,71.00,1925.0,14.0,79,2,vw rabbit custom
2,9,19.0,6,156.0,108.0,2930.0,15.5,76,3,toyota mark ii
3,11,28.0,4,90.0,75.00,2125.0,14.5,74,1,dodge colt
4,13,37.7,4,89.0,62.00,2050.0,17.3,81,3,toyota tercel


In [39]:
# 特徴変数
X = df_train.iloc[:,2:]
# 目的変数
Y = df_train['mpg']

In [40]:
X.head()

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,4,135.0,84.00,2525.0,16.0,82,1,dodge aries se
1,4,89.0,71.00,1925.0,14.0,79,2,vw rabbit custom
2,6,156.0,108.0,2930.0,15.5,76,3,toyota mark ii
3,4,90.0,75.00,2125.0,14.5,74,1,dodge colt
4,4,89.0,62.00,2050.0,17.3,81,3,toyota tercel


In [41]:
# 統計情報
X.describe()

,cylinders,displacement,weight,acceleration,model year,origin
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,5.296482,183.311558,2883.839196,15.647739,76.165829,1.582915
std,1.644562,98.400457,819.766870,2.701885,3.802928,0.798932
min,3.000000,71.000000,1613.000000,8.500000,70.000000,1.000000
25%,4.000000,98.000000,2217.500000,14.000000,73.000000,1.000000
50%,4.000000,140.000000,2702.000000,15.500000,76.000000,1.000000
75%,6.000000,250.000000,3426.500000,17.150000,80.000000,2.000000
max,8.000000,454.000000,5140.000000,23.700000,82.000000,3.000000


In [42]:
# 欠損値確認
X.isnull().sum()

cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [43]:
# 'car name'について同じ車種の出現回数のカウントのサマリ
X['car name'].value_counts().value_counts()

1    143
2     18
3      4
4      2
Name: car name, dtype: int64

In [44]:
# どこかに？の文字列があると怒られたので処理する
for column in X.columns:
    for v in X[column]:
        if (v == "?"):
            print(column)

horsepower
horsepower
horsepower
horsepower


In [55]:
X.query('horsepower=="?"')

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
24,4,151.0,?,3035.0,20.5,82,1,amc concord dl
113,4,100.0,?,2320.0,15.8,81,2,renault 18i
145,4,85.0,?,1835.0,17.3,80,2,renault lecar deluxe
175,4,98.0,?,2046.0,19.0,71,1,ford pinto


In [62]:
# 一旦削除してみる
drop_list = X.query('horsepower=="?"').index
X_drop = X.drop(drop_list)
X_drop.query('horsepower=="?"')

,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name


In [63]:
Y_drop = Y.drop(drop_list)

In [64]:
# 車種も除外してみる
X_drop = X_drop.iloc[:,:-1]

In [65]:
X_drop.head()

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,4,135.0,84.00,2525.0,16.0,82,1
1,4,89.0,71.00,1925.0,14.0,79,2
2,6,156.0,108.0,2930.0,15.5,76,3
3,4,90.0,75.00,2125.0,14.5,74,1
4,4,89.0,62.00,2050.0,17.3,81,3


In [66]:
# 学習用とテスト用に分割する
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_drop, Y_drop, test_size = 0.3)

In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# n_estimators:作成する決定木の数 random_state:ランダム性を固定する数値 n_jobs:使用するCPU数 -1はすべて
reg_rf = RandomForestRegressor(n_estimators=1000, random_state=1, n_jobs=-1)
# 適用する
reg_rf.fit(X=X_train, y=y_train)

# 決定係数の算出
reg_rf.score(X=X_test, y=y_test)
# 予測値の算出
test_pred = reg_rf.predict(X_test)

# MES平均二乗誤差の算出
test_mse = mean_squared_error(y_true=y_test, y_pred=test_pred)

print(test_mse)
# RMES　MESの平方根
np.sqrt(test_mse)

6.323698887627058


2.514696579634819

In [72]:
# 実測値の平均をとる
np.mean(y_test)

22.518644067796615

実測値の平均が22.5に対して、RMESが2.5 → 誤差は小さい？

In [114]:
# 解答を作成
df_test = pd.read_csv('test.tsv',delimiter='\t' )

In [115]:
# ? があるか確認
df_test.query('horsepower=="?"')

,id,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
70,138,6,200.0,?,2875.0,17.0,74,1,ford maverick
112,221,4,140.0,?,2905.0,14.3,80,1,ford mustang cobra


In [116]:
# 平均値で埋めてみる
horsepower = df_test.query('horsepower!="?"')["horsepower"]
horsepower = horsepower.astype(float)
horsepower_mean = horsepower.mean()
horsepower_mean

107.60913705583756

In [117]:
list = df_test.query('horsepower=="?"').index
df_test.loc[list, 'horsepower'] = horsepower_mean

In [118]:
df_test.query('horsepower=="?"')

,id,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name


In [119]:
test_X = df_test.iloc[:,1:-1]

In [120]:
test_X.head()

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,6,145.0,76.00,3160.0,19.6,81,2
1,6,250.0,98.00,3525.0,19.0,77,1
2,4,119.0,92.00,2434.0,15.0,80,3
3,6,258.0,110.0,2962.0,13.5,71,1
4,4,97.0,88.00,2100.0,16.5,72,3


In [121]:
ans = reg_rf.predict(test_X)
ans

array([28.3185, 17.4915, 30.5728, 19.205 , 26.8641, 25.1186, 15.2845,
       18.6942, 25.2774, 21.8239, 12.648 , 26.9999, 13.1765, 33.6507,
       34.8896, 15.2178, 23.9317, 24.5377, 14.6755, 29.6561, 25.3321,
       41.9277, 18.82  , 13.114 , 14.0872, 13.0845, 14.677 , 21.8811,
       28.7298, 28.3195, 35.2235, 20.903 , 38.3815, 12.2965, 20.8379,
       30.9461, 33.2502, 21.2417, 27.9503, 38.7663, 12.0855, 28.8208,
       21.2593, 30.7085, 20.7576, 38.9755, 34.8153, 20.3887, 24.2151,
       23.7255, 25.379 , 25.5318, 33.4647, 18.9691, 27.2138, 15.1381,
       15.4643, 31.9449, 15.4876, 16.313 , 15.4902, 26.2846, 31.1326,
       17.2449, 27.1981, 14.1255, 38.3026, 29.0307, 19.4816, 14.9542,
       19.6673, 28.5391, 33.7726, 12.1265, 35.2455, 27.7012, 20.7389,
       15.0056, 13.5405, 17.6934, 31.4888, 33.5234, 23.9374, 26.5651,
       24.0171, 38.5126, 17.8397, 16.1843, 14.8425, 15.4876, 22.5427,
       31.159 , 13.84  , 22.1936, 20.5589, 13.4931, 34.0135, 26.4588,
       12.357 , 25.2

In [122]:
# 提出用に加工
np_id = df_test['id'].values
dd=pd.DataFrame({"id":np_id, "ans":ans})
dd.to_csv("result01.csv",header=False,index=False)

結果は"2.86565"でした　(値が0に近いほどよい)

In [125]:
# 重要度を確認
importances = pd.DataFrame(X_drop.columns, columns=['label'])
importances['importrances'] = reg_rf.feature_importances_
importances = importances.sort_values('importrances', ascending=False)
importances

,label,importrances
2,horsepower,0.333386
3,weight,0.240960
1,displacement,0.208077
5,model year,0.157587
4,acceleration,0.030710
0,cylinders,0.025386
6,origin,0.003895
